# Алгоритмы Витерби

In [1]:
import numpy as np

In [2]:
def open_state_to_index(state):
    mapping = {'О': 0, 'Р': 1}
    return mapping[state]

def index_to_hidden_state(idx):
    mapping = [1, 2]
    return mapping[idx]

In [3]:
def viterbi(open_states, P, A, B):
    assert len(P) == A.shape[0] == B.shape[0]
    psi = []
    idx = open_state_to_index(open_states[0])
    epsilon = (P * B[:, idx]).reshape(-1, 1)
    for i, open_state in enumerate(open_states[1:]):
        idx = open_state_to_index(open_state)
        psi.append(np.argmax(A * epsilon, axis=0))
        epsilon = (np.max(A * epsilon, axis=0) * B[:, idx]).reshape(-1, 1)
    hidden_states = []
    hidden_states.append(np.argmax(epsilon))
    n = len(psi)
    for i in range(n):
        hidden_states.append(psi[n - i - 1][hidden_states[i - 1]])
    hidden_states.reverse()
    return [index_to_hidden_state(idx) for idx in hidden_states]

# Алгоритм Forward-Backward

In [4]:
def forward_backward(open_states, P, A, B):
    assert len(P) == A.shape[0] == B.shape[0]
    n = len(open_states)
    alphas = []
    idx = open_state_to_index(open_states[0])
    alpha = (P * B[:, idx]).reshape(-1, 1)
    alphas.append(alpha)
    for i, open_state in enumerate(open_states[1:]):
        idx = open_state_to_index(open_state)
        alpha = (A * B[:, idx].reshape(-1, 1)) @ alpha
        alphas.append(alpha)
    alphas = np.array(alphas).squeeze()
    
    betas = []
    beta = np.ones_like(P).reshape(-1, 1)
    betas.append(beta)
    for i in range(n - 1):
        idx = open_state_to_index(open_states[n - i - 1])
        beta = (A.T * B[:, idx].reshape(1, -1)) @ beta
        betas.append(beta)
    betas.reverse()
    betas = np.array(betas).squeeze()
    probs = alphas * betas / alphas[-1].sum()
    return probs

## Тестирование

In [5]:
open_states = 'ОРОРОРООРРРРРРРРРРОООООООО'

In [6]:
P = np.array([0.5, 0.5], dtype=np.float128)
P

array([0.5, 0.5], dtype=float128)

## Тест 1

In [7]:
B = np.array(
    [
        [0.5, 0.5],
        [0.1, 0.9]
    ], dtype=np.float128
)
B

array([[0.5, 0.5],
       [0.1, 0.9]], dtype=float128)

In [8]:
A = np.array(
    [
        [0.8, 0.2],
        [0.2, 0.8]
    ], dtype=np.float128
)
A

array([[0.8, 0.2],
       [0.2, 0.8]], dtype=float128)

In [9]:
viterbi(open_states, P, A, B)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
hidden_state_probs = forward_backward(open_states, P, A, B)
hidden_state_probs

array([[0.85956173, 0.14043827],
       [0.76609555, 0.23390445],
       [0.8783159 , 0.1216841 ],
       [0.77939405, 0.22060595],
       [0.88686798, 0.11313202],
       [0.79512375, 0.20487625],
       [0.91061422, 0.08938578],
       [0.85999559, 0.14000441],
       [0.45699127, 0.54300873],
       [0.27318414, 0.72681586],
       [0.18984842, 0.81015158],
       [0.15315539, 0.84684461],
       [0.13940407, 0.86059593],
       [0.13971214, 0.86028786],
       [0.15427858, 0.84572142],
       [0.19251224, 0.80748776],
       [0.27910923, 0.72089077],
       [0.47000478, 0.52999522],
       [0.88850328, 0.11149672],
       [0.96461882, 0.03538118],
       [0.97845654, 0.02154346],
       [0.98093931, 0.01906069],
       [0.98120371, 0.01879629],
       [0.98022273, 0.01977727],
       [0.97438637, 0.02561363],
       [0.94221681, 0.05778319]], dtype=float128)

In [11]:
# сумма вероятностей скрытых состояний в каждый момент времени t равна 1
hidden_state_probs.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float128)

In [12]:
list(np.argmax(hidden_state_probs, axis=1) + 1)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]

## Тест 2

In [13]:
B = np.array(
    [
        [0.5,  0.5],
        [0.51,  0.49]
    ], dtype=np.float128
)
B

array([[0.5 , 0.5 ],
       [0.51, 0.49]], dtype=float128)

In [14]:
A = np.array(
    [
        [0.5, 0.5],
        [0.5, 0.5]
    ], dtype=np.float128
)
A

array([[0.5, 0.5],
       [0.5, 0.5]], dtype=float128)

In [15]:
viterbi(open_states, P, A, B)

[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]

In [16]:
hidden_state_probs = forward_backward(open_states, P, A, B)
hidden_state_probs

array([[0.4950495 , 0.5049505 ],
       [0.50505051, 0.49494949],
       [0.4950495 , 0.5049505 ],
       [0.50505051, 0.49494949],
       [0.4950495 , 0.5049505 ],
       [0.50505051, 0.49494949],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.50505051, 0.49494949],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ],
       [0.4950495 , 0.5049505 ]], dtype=float128)

In [17]:
# сумма вероятностей скрытых состояний в каждый момент времени t равна 1
hidden_state_probs.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float128)

In [18]:
list(np.argmax(hidden_state_probs, axis=1) + 1)

[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]